In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch


C:\Users\MALAV\AppData\Local\Temp\ipykernel_31992\896286104.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
d:\8th sem project\Semantic search - elastic\elasticsearch-8.12.1-windows-x86_64\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
excel_file_path = "history_cleaned_file.csv"
df = pd.read_csv(excel_file_path, encoding='utf-8')


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


tokenizer_config.json: 100%|██████████| 48.0/48.0 [00:00<00:00, 13.3kB/s]
d:\8th sem project\Semantic search - elastic\elasticsearch-8.12.1-windows-x86_64\venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MALAV\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.wa

In [ ]:
encoded_descriptions = []
for description in df['Description'].tolist():
    encoded_input = tokenizer(description, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**encoded_input)
        embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0)
        encoded_descriptions.append(embeddings.numpy())


In [ ]:
encoded_descriptions_tensors = [torch.from_numpy(arr) for arr in encoded_descriptions]
embedding_array = torch.stack(encoded_descriptions_tensors).numpy()
embedding_df = pd.DataFrame(embedding_array)

In [ ]:
embedding_df.to_csv('embeddings_history.csv', index=False)


In [ ]:
def dense_retrieval_bert(query, descriptions, tokenizer, model):
    encoded_query = tokenizer(query, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        query_outputs = model(**encoded_query)
        query_embedding = torch.mean(query_outputs.last_hidden_state, dim=1).squeeze(0)
        similarities = torch.nn.functional.cosine_similarity(query_embedding, descriptions)
        sorted_indices = similarities.argsort(descending=True)
        return sorted_indices


In [ ]:
query = "swords"
descriptions_tensor = torch.tensor(embedding_array)
sorted_indices = dense_retrieval_bert(query, descriptions_tensor, tokenizer, model)


tensor([ 4484, 10498,  1688,  ...,  5189,   153,   377])

0        History helps us understand how early humans a...
1        Historians assess various situations over a lo...
2        Historians aim to verify facts from myths root...
3        Ancient Indian history was recorded through a ...
4        Archaeologist John Marshall identified the old...
                               ...                        
11436    Learners will understand the forms of currency...
11437    Irrigation methods included rain, wells, river...
11438    Important water reservoirs in medieval Rajasth...
11439    Various devices, including pulleys, levers, an...
11440    Firuz Shah Tughlaq laid the largest network of...
Name: Description, Length: 11441, dtype: object
0        History helps us understand how early humans a...
1        Historians assess various situations over a lo...
2        Historians aim to verify facts from myths root...
3        Ancient Indian history was recorded through a ...
4        Archaeologist John Marshall identified the old...
        

In [ ]:
excel_file_path = "merged_chemistry_history.csv"
df = pd.read_csv(excel_file_path)
df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.094657,-0.122383,-0.079896,-0.044538,-0.007379,0.090962,0.047580,0.077806,0.020454,0.285754,...,0.031754,-0.076722,0.078073,-0.029192,-0.111309,-0.447907,-0.059144,0.121249,-0.096674,-0.066743
1,0.081634,-0.129273,-0.026265,0.041838,0.042232,0.184493,0.009504,0.091468,-0.004399,0.053288,...,-0.067308,-0.085021,-0.067374,-0.011594,-0.008629,-0.197608,-0.044792,0.047884,0.042787,-0.062521
2,0.086238,-0.298992,-0.100434,0.180817,-0.015123,-0.044194,-0.019331,-0.062212,0.021878,0.280359,...,0.049362,-0.079262,-0.012248,-0.012059,-0.017202,-0.204966,-0.031667,0.012707,-0.043380,0.026171
3,0.072963,-0.002325,-0.080909,0.083731,-0.006488,0.044944,-0.001377,0.085474,-0.046180,0.112794,...,0.100325,-0.059208,-0.029511,0.030759,-0.073102,-0.270272,-0.065157,0.073536,-0.243473,-0.057759
4,0.031103,-0.034055,-0.093266,0.114449,-0.044906,0.054335,-0.034160,0.011134,0.068368,0.019083,...,0.238694,-0.059263,0.095135,0.029214,-0.088667,-0.296308,-0.050580,0.005183,0.009428,0.029314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13177,-0.061179,0.064019,-0.105568,0.270328,0.033692,-0.062505,0.192990,-0.037999,0.031764,-0.064538,...,-0.045069,0.116942,0.156994,-0.014287,0.014957,-0.038206,-0.030560,0.180730,-0.114854,0.127229
13178,-0.107158,-0.007713,-0.099509,-0.052122,0.081469,-0.005424,0.309784,-0.129926,0.327487,-0.092276,...,0.049573,0.072928,-0.042977,-0.033741,0.149205,-0.086706,0.009554,-0.009855,-0.017961,0.051693
13179,-0.098555,-0.246992,-0.043319,0.000650,0.030874,-0.046191,0.101342,-0.036918,0.072136,-0.059903,...,-0.002850,0.111143,-0.120164,0.076012,0.057087,-0.357373,-0.008265,0.026785,-0.055839,0.044525
13180,-0.039613,-0.136299,0.009743,-0.002164,-0.061060,0.030613,0.326696,-0.117655,0.187947,-0.077179,...,-0.109736,-0.067487,0.069684,0.040600,0.097760,-0.097576,-0.043346,0.081766,-0.180143,-0.049991


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import pandas as pd

excel_file_path = "mergedactual_chemistry_history.csv"
df2 = pd.read_csv(excel_file_path)
df2

,Description,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,What is the atomic number of neon? The atomic ...,0.094657,-0.122383,-0.079896,-0.044538,-0.007379,0.090962,0.047580,0.077806,0.020454,...,0.031754,-0.076722,0.078073,-0.029192,-0.111309,-0.447907,-0.059144,0.121249,-0.096674,-0.066743
1,Why are Times Square and Las Vegas said to be ...,0.081634,-0.129273,-0.026265,0.041838,0.042232,0.184493,0.009504,0.091468,-0.004399,...,-0.067308,-0.085021,-0.067374,-0.011594,-0.008629,-0.197608,-0.044792,0.047884,0.042787,-0.062521
2,How can you identify a tube filled with neon? ...,0.086238,-0.298992,-0.100434,0.180817,-0.015123,-0.044194,-0.019331,-0.062212,0.021878,...,0.049362,-0.079262,-0.012248,-0.012059,-0.017202,-0.204966,-0.031667,0.012707,-0.043380,0.026171
3,"Can 'neon' lights show other colors? If so, wh...",0.072963,-0.002325,-0.080909,0.083731,-0.006488,0.044944,-0.001377,0.085474,-0.046180,...,0.100325,-0.059208,-0.029511,0.030759,-0.073102,-0.270272,-0.065157,0.073536,-0.243473,-0.057759
4,What unique characteristic helps to identify g...,0.031103,-0.034055,-0.093266,0.114449,-0.044906,0.054335,-0.034160,0.011134,0.068368,...,0.238694,-0.059263,0.095135,0.029214,-0.088667,-0.296308,-0.050580,0.005183,0.009428,0.029314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13177,Learners will understand the forms of currency...,-0.061179,0.064019,-0.105568,0.270328,0.033692,-0.062505,0.192990,-0.037999,0.031764,...,-0.045069,0.116942,0.156994,-0.014287,0.014957,-0.038206,-0.030560,0.180730,-0.114854,0.127229
13178,"Irrigation methods included rain, wells, river...",-0.107158,-0.007713,-0.099509,-0.052122,0.081469,-0.005424,0.309784,-0.129926,0.327487,...,0.049573,0.072928,-0.042977,-0.033741,0.149205,-0.086706,0.009554,-0.009855,-0.017961,0.051693
13179,Important water reservoirs in medieval Rajasth...,-0.098555,-0.246992,-0.043319,0.000650,0.030874,-0.046191,0.101342,-0.036918,0.072136,...,-0.002850,0.111143,-0.120164,0.076012,0.057087,-0.357373,-0.008265,0.026785,-0.055839,0.044525
13180,"Various devices, including pulleys, levers, an...",-0.039613,-0.136299,0.009743,-0.002164,-0.061060,0.030613,0.326696,-0.117655,0.187947,...,-0.109736,-0.067487,0.069684,0.040600,0.097760,-0.097576,-0.043346,0.081766,-0.180143,-0.049991


In [ ]:
def bert_search(query, model, description_embeddings):
    tokenized_query = tokenizer(query, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        output = model(**tokenized_query)
    query_embedding = output.last_hidden_state.mean(dim=1)
    description_embeddings_tensor = torch.tensor(description_embeddings.values)
    similarities = torch.nn.functional.cosine_similarity(query_embedding, description_embeddings_tensor)
    sorted_indices = similarities.argsort(descending=True)
    return sorted_indices


In [ ]:
import time

In [ ]:
matrix1 = []
query = "Hindu and muslim religion"
sorted_indices = bert_search(query, model, df)
for idx in sorted_indices[:10]:
    matrix1.append(df['0'][idx.item()])

selected_descriptions = df2[df2['0'].isin(matrix1)]['Description']
selected_descriptions

TypeError: bert_search() missing 1 required positional argument: 'description_embeddings'